# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key 


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,blackmans bay,-43.0167,147.3167,284.22,60,93,1.34,AU,1727217825
1,1,adamstown,-25.0660,-130.1015,292.72,70,86,6.63,PN,1727217826
2,2,kone,-21.0595,164.8658,302.19,41,1,2.89,NC,1727217828
3,3,iqaluit,63.7506,-68.5145,279.00,86,75,3.09,CA,1727217829
4,4,waitangi,-43.9535,-176.5597,281.94,78,16,6.26,NZ,1727217830


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng',  
    'Lat',  
    geo=True,  
    tiles='OSM',  
    size='Humidity',  
    color='Humidity',  
    alpha=0.5,  
    title='City Humidity Levels',  
    width=800,  
    height=500  
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] < 305) & (city_data_df['Max Temp'] > 290)
]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,adamstown,-25.0660,-130.1015,292.72,70,86,6.63,PN,1727217826
2,2,kone,-21.0595,164.8658,302.19,41,1,2.89,NC,1727217828
6,6,west island,-12.1568,96.8225,299.14,94,75,6.17,CC,1727217832
7,7,luba,3.4568,8.5547,298.10,80,100,0.49,GQ,1727217834
9,9,ribeira grande,38.5167,-28.7000,297.47,92,100,8.49,PT,1727217836


### Step 3: Create a new DataFrame called `hotel_df`.

In [9]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
1,adamstown,PN,-25.0660,-130.1015,70,
2,kone,NC,-21.0595,164.8658,41,
6,west island,CC,-12.1568,96.8225,94,
7,luba,GQ,3.4568,8.5547,80,
9,ribeira grande,PT,38.5167,-28.7000,92,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [24]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'apiKey': geoapify_key, 
    'radius': radius,
    'limit': 1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"category:hotel"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary 
    response = requests.get(base_url, params=params)
    name_address = response.json()



    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
adamstown - nearest hotel: No hotel found
kone - nearest hotel: No hotel found
west island - nearest hotel: No hotel found
luba - nearest hotel: No hotel found
ribeira grande - nearest hotel: No hotel found
alotau - nearest hotel: No hotel found
paracuru - nearest hotel: No hotel found
rio grande - nearest hotel: No hotel found
hawaiian paradise park - nearest hotel: No hotel found
pacific grove - nearest hotel: No hotel found
guane - nearest hotel: No hotel found
san antonio de pale - nearest hotel: No hotel found
las tablas - nearest hotel: No hotel found
saipan - nearest hotel: No hotel found
wailua homesteads - nearest hotel: No hotel found
tazacorte - nearest hotel: No hotel found
alice springs - nearest hotel: No hotel found
hasaki - nearest hotel: No hotel found
sibiti - nearest hotel: No hotel found
lihue - nearest hotel: No hotel found
fisterra - nearest hotel: No hotel found
nguigmi - nearest hotel: No hotel found
east london - nearest hotel: No hotel fo

,City,Country,Lat,Lng,Humidity,Hotel Name
1,adamstown,PN,-25.0660,-130.1015,70,No hotel found
2,kone,NC,-21.0595,164.8658,41,No hotel found
6,west island,CC,-12.1568,96.8225,94,No hotel found
7,luba,GQ,3.4568,8.5547,80,No hotel found
9,ribeira grande,PT,38.5167,-28.7000,92,No hotel found
...,...,...,...,...,...,...
552,cabedelo,BR,-6.9811,-34.8339,79,No hotel found
555,kuta,ID,-8.7233,115.1723,94,No hotel found
556,lobito,AO,-12.3481,13.5456,86,No hotel found
558,mogadishu,SO,2.0371,45.3438,81,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [26]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    'Lng',  
    'Lat',  
    geo=True,  
    tiles='OSM',  
    size='Humidity',  
    color='Humidity',  
    alpha=0.5,  
    title='City Humidity Levels and Nearby Hotels',  
    width=800,  
    height=500,
    hover_cols=['Hotel Name', 'Country'] 
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hotel Name,Country)